# Guided Capstone Step 4 - Pre-processing and Training Data Development

### **The Data Science Method**  


1.   Problem Identification 

2.   Data Wrangling 
  
3.   Exploratory Data Analysis 
 
4.   **Pre-processing and Training Data Development**

5.  Modeling
  * Fit Models with Training Data Set
  * Review Model Outcomes — Iterate over additional models as needed.
  * Identify the Final Model

6.   Documentation
  * Review the Results
  * Present and share your findings - storytelling
  * Finalize Code 
  * Finalize Documentation

In this step of the guided capstone, you'll revisit some of the steps you completed to solve questions in step 4. Revisiting these steps will help you get additional practice completing this kind of work.

**<font color='teal'> Start by loading the necessary packages, as you've done in the previous steps. Print out your current working directory to confirm that you are in the correct project directory. </font>**

In [187]:
#load python packages
import os
import pandas as pd
import datetime
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

**<font color='teal'>  Load the csv file you created in step 3. Remember, it should be saved inside your data subfolder. Print the first five rows.</font>**

In [188]:
df=pd.read_csv('/resources/springboard/GuidedCapstone/data/step3_output.csv',index_col=False)
df.head(5)

,Unnamed: 0,Unnamed: 0.1,Name,state,summit_elev,vertical_drop,base_elev,trams,fastEight,fastSixes,...,LongestRun_mi,SkiableTerrain_ac,Snow Making_ac,daysOpenLastYear,yearsOpen,averageSnowfall,AdultWeekday,AdultWeekend,projectedDaysOpen,NightSkiing_ac
0,0,0,Alyeska Resort,Alaska,3939,2500,250,1,0.0,0,...,1.0,1610.0,113.0,150.0,60.0,669.0,65.0,85.0,150.0,550.0
1,1,1,Eaglecrest Ski Area,Alaska,2600,1540,1200,0,0.0,0,...,2.0,640.0,60.0,45.0,44.0,350.0,47.0,53.0,90.0,0.0
2,2,2,Hilltop Ski Area,Alaska,2090,294,1796,0,0.0,0,...,1.0,30.0,30.0,150.0,36.0,69.0,30.0,34.0,152.0,30.0
3,3,3,Arizona Snowbowl,Arizona,11500,2300,9200,0,0.0,1,...,2.0,777.0,104.0,122.0,81.0,260.0,89.0,89.0,122.0,0.0
4,4,4,Sunrise Park Resort,Arizona,11100,1800,9200,0,0.0,0,...,1.2,800.0,80.0,115.0,49.0,250.0,74.0,78.0,104.0,80.0


In [189]:
df_name=df.Name

In [190]:
df=df.drop(columns=['Name', 'Unnamed: 0', 'Unnamed: 0.1'])
df.head(5)

,state,summit_elev,vertical_drop,base_elev,trams,fastEight,fastSixes,fastQuads,quad,triple,...,LongestRun_mi,SkiableTerrain_ac,Snow Making_ac,daysOpenLastYear,yearsOpen,averageSnowfall,AdultWeekday,AdultWeekend,projectedDaysOpen,NightSkiing_ac
0,Alaska,3939,2500,250,1,0.0,0,2,2,0,...,1.0,1610.0,113.0,150.0,60.0,669.0,65.0,85.0,150.0,550.0
1,Alaska,2600,1540,1200,0,0.0,0,0,0,0,...,2.0,640.0,60.0,45.0,44.0,350.0,47.0,53.0,90.0,0.0
2,Alaska,2090,294,1796,0,0.0,0,0,0,1,...,1.0,30.0,30.0,150.0,36.0,69.0,30.0,34.0,152.0,30.0
3,Arizona,11500,2300,9200,0,0.0,1,0,2,2,...,2.0,777.0,104.0,122.0,81.0,260.0,89.0,89.0,122.0,0.0
4,Arizona,11100,1800,9200,0,0.0,0,1,2,3,...,1.2,800.0,80.0,115.0,49.0,250.0,74.0,78.0,104.0,80.0


## Create dummy features for categorical variables

**<font color='teal'> Create dummy variables for `State`. Add the dummies back to the dataframe and remove the original column for `State`. </font>**

In [191]:
dfo=df.select_dtypes(include=['object']) # select object type columns
df = pd.concat([df.drop(dfo, axis=1), pd.get_dummies(dfo)], axis=1)
df.head(3)

,summit_elev,vertical_drop,base_elev,trams,fastEight,fastSixes,fastQuads,quad,triple,double,...,state_Rhode Island,state_South Dakota,state_Tennessee,state_Utah,state_Vermont,state_Virginia,state_Washington,state_West Virginia,state_Wisconsin,state_Wyoming
0,3939,2500,250,1,0.0,0,2,2,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2600,1540,1200,0,0.0,0,0,0,0,4,...,0,0,0,0,0,0,0,0,0,0
2,2090,294,1796,0,0.0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


## Standardize the magnitude of numeric features

**<font color='teal'> Using sklearn preprocessing, standardize the scale of the features of the dataframe except the name of the resort, which you don't need in the dataframe for modeling so it can be droppped here as well. You should  hold out your response variable(s) so that you can have their true values available for model performance review. Let's set `AdultWeekend` to the y variable as your response for scaling and modeling. Later, you will go back and consider the `AdultWeekday`, `dayOpenLastYear`, and `projectedDaysOpen`. For now leave, them in the development dataframe. </font>**

In [192]:
# hold out the response variables for later use
responseV=['AdultWeekend','AdultWeekday','daysOpenLastYear','projectedDaysOpen']
featureV=list(set(df.columns)-set(responseV))
df_response=df[responseV]
df_feature=df[featureV]
df_response.head(2)

,AdultWeekend,AdultWeekday,daysOpenLastYear,projectedDaysOpen
0,85.0,65.0,150.0,150.0
1,53.0,47.0,45.0,90.0


In [193]:
# all numeric data scaled to 0 mean and unit std
from sklearn import preprocessing
X = df_feature.values
y = df_response['AdultWeekend'].values
X_scale=preprocessing.scale(X)
y_scale=preprocessing.scale(y)

## Split into training and testing datasets

**<font color='teal'> Using sklearn model selectionm import train_test_split, and create a 75/25 split with the y = `AdultWeekend`. You will start by using the adult weekend ticket price as your response variable for modeling.</font>**

In [194]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_scale, y_scale, test_size=0.25, random_state=0)

Here, you'll start the actual modeling work. First let's fit a multiple linear regression model to predict the `AdultWeekend` price.

# Guided Capstone Step 5 - Modeling

You've cleaned and prepared the datasets, so now it's time to get into the most exciting part of this process: modeling! In this exercise, you will build three different models and compare each model's performance. In the end, you will choose the best model for proving insights to Big Mountain management.  

1.   Problem Identification 

2.   Data Wrangling 
  
3.   Exploratory Data Analysis 
 
4.   Pre-processing and Training Data Development

5.   **Modeling**
  * Fit Models with Training Data Set
  * Review Model Outcomes — Iterate over additional models as needed.
  * Identify the Final Model

6.   Documentation
  * Review the Results
  * Present and share your findings - storytelling
  * Finalize Code 
  * Finalize Documentation

## Fit Models with Training Data Set

**<font color='teal'> Using sklearn, fit the model on your training dataset.</font>**

#### Model 1

In [195]:
#all first model set
from sklearn import linear_model
from sklearn.metrics import r2_score,mean_absolute_error
lm = linear_model.LinearRegression()
model = lm.fit(X_train,y_train)

**<font color='teal'> Predict on the testing dataset and score the model performance with the y_test set and the y-pred values. The explained variance is a measure of the variation explained by the model. This is also known as the R-squared value. </font>**

In [196]:
# prediction
y_pred=lm.predict(X_test)
# R-squared value
print('r2_score for model 1: ',r2_score(y_test, y_pred))

r2_score for model 1:  -8.483096633956736e+24


In [197]:
# mean_absolute_error
print('mean_absolute_error for model 1: ',mean_absolute_error(y_test, y_pred))

mean_absolute_error for model 1:  362070686447.76843


In [198]:
print(explained_variance_score(y_test, y_pred))

-8.350471682716526e+24


## Review Model Outcomes — Iterate over additional models as needed.

**<font color='teal'> Print the intercept value from the linear model. </font>**

In [199]:
print(lm.intercept_)

91066263560.93335


**<font color='teal'> The intercept is the mean `AdultWeekend` price for all the resorts given the other characteristics. The addition or subtraction of each of the coefficient values in the regression are numeric adjustments applied to the intercept to provide a particular observation's value for the `AdultWeekend` resulting value. Also, because you took the time to scale your x values in the training data, you can compare each of the coefficients for the features to determine the feature's importance. Print the coefficient values from the linear model and sort in descending order to identify the top ten most important features.</font>** 

Hint: make sure to review the absolute value of the coefficients, because the adjustment may be positive or negative, but what you are looking for is the magnitude of impact on your response variable.

In [200]:
coeff=lm.coef_
print(coeff)

[-6.52878121e+07  3.34307075e-01 -3.27937641e+07 -1.24277991e+08
  6.69334065e+11 -1.48826415e+08  1.45382175e+12  2.06493251e+11
 -1.45640515e+08 -1.11685683e+08 -1.59512997e-01 -4.63068500e+07
 -1.20254373e+08  8.10672312e+11 -3.27937641e+07 -1.16063223e+08
 -4.63068501e+07 -1.28149860e+08 -4.63068500e+07 -5.27034160e-02
  2.18238177e+11 -5.66275567e+07 -1.07098898e+08 -3.32014834e-02
 -3.33035212e-02 -2.13840013e+12 -5.66275567e+07  2.72942634e+00
 -6.52878122e+07 -9.71778062e+07 -9.17627140e+07 -7.28819531e+07
 -2.62878051e+00  4.39453125e-02 -1.02274722e+08 -4.63068499e+07
 -1.68917858e+08 -6.52878120e+07  5.45196533e-02  2.40324230e+11
  7.59539165e+11  4.83920935e+11 -1.11685683e+08 -7.97152568e+07
  1.23046875e-01 -1.38979362e+08 -4.63068501e+07 -7.28819530e+07
  1.13488770e+00 -1.28149860e+08 -9.71778064e+07 -6.52878121e+07
 -1.78989895e+08 -1.02274722e+08  5.97091968e+11]


In [201]:
df_res1=pd.DataFrame()
df_res1['feature']=df_feature.columns
df_res1['values']=abs(lm.coef_)
df_res1.sort_values(by=['values'],ascending=False)

,feature,values
25,total_chairs,2.138400e+12
6,fastEight,1.453822e+12
13,fastQuads,8.106723e+11
40,surface,7.595392e+11
4,double,6.693341e+11
54,triple,5.970920e+11
41,quad,4.839209e+11
39,fastSixes,2.403242e+11
20,state_Rhode Island,2.182382e+11
7,trams,2.064933e+11


**<font color='teal'>You should see that the top ten important features are different states. However, the state is not something the managers at the Big Mountain Resort can do anything about. Given that you care more about actionable traits associated with ticket pricing, rebuild the model without the state features and compare the results. </font>**

#### Model 2

In [202]:
# remove 'state' columns
df_feature2=df_feature.drop(df_feature.filter(regex='state').columns, axis=1)
X = df_feature2.values
# scale features
X_scale=preprocessing.scale(X)
#train/test split
X_train, X_test, y_train, y_test = train_test_split(X_scale, y_scale, test_size=0.25, random_state=0)
# model fit
lm = linear_model.LinearRegression()
model = lm.fit(X_train,y_train)


In [203]:
df_feature2.head(2)

,Runs,double,fastEight,trams,Snow Making_ac,fastQuads,SkiableTerrain_ac,LongestRun_mi,TerrainParks,total_chairs,base_elev,summit_elev,yearsOpen,averageSnowfall,fastSixes,surface,quad,NightSkiing_ac,vertical_drop,triple
0,76.0,0,0.0,1,113.0,2,1610.0,1.0,2.0,7,250,3939,60.0,669.0,0,2,2,550.0,2500,0
1,36.0,4,0.0,0,60.0,0,640.0,2.0,1.0,4,1200,2600,44.0,350.0,0,0,0,0.0,1540,0


In [204]:
# prediction
y_pred=lm.predict(X_test)
# R-squared value
print('r2_score for model 2: ',r2_score(y_test, y_pred))

# mean_absolute_error
print('mean_absolute_error for model 2: ',mean_absolute_error(y_test, y_pred))

print(lm.intercept_)
print(lm.coef_)
print(explained_variance_score(y_test, y_pred))

r2_score for model 2:  0.357228244923836
mean_absolute_error for model 2:  0.5778803285137045
0.006189937699032659
[ 1.34896212e-01  7.33686237e-02  1.16573418e-15 -1.18866575e-01
 -6.32659599e-02  1.54600993e-01 -3.55470980e-02  5.68817923e-02
  2.59648368e-02  9.03235414e-02 -2.80349963e-01  3.40040422e-01
  4.65742672e-02 -4.79181654e-02 -5.73283755e-02 -1.54607551e-01
  2.18797329e-01  5.01454044e-03  3.08273736e-01  1.14859081e-01]
0.35784099413004256


In [205]:
df_res2=pd.DataFrame()
df_res2['feature']=df_feature2.columns
df_res2['values']=abs(lm.coef_)
df_res2.sort_values(by=['values'],ascending=False)

,feature,values
11,summit_elev,3.400404e-01
18,vertical_drop,3.082737e-01
10,base_elev,2.803500e-01
16,quad,2.187973e-01
15,surface,1.546076e-01
5,fastQuads,1.546010e-01
0,Runs,1.348962e-01
3,trams,1.188666e-01
19,triple,1.148591e-01
9,total_chairs,9.032354e-02


**<font color='teal'> When reviewing your new model coefficients, note that `summit_elev` is now in the number two spot. From a managerial perspective, this is also difficult to change and highly correlated with `base_elev` and `vertical_drop`.  This time rebuild the model without the state features and without the `summit_elev` and without `base_elev`and compare the results. </font>**

#### Model 3

In [206]:
# remove 'summit_elev' and 'base_elev' columns
to_drop=['summit_elev','base_elev']
df_feature3=df_feature2.drop(to_drop,axis=1)

In [207]:
df_feature3.head(2)

,Runs,double,fastEight,trams,Snow Making_ac,fastQuads,SkiableTerrain_ac,LongestRun_mi,TerrainParks,total_chairs,yearsOpen,averageSnowfall,fastSixes,surface,quad,NightSkiing_ac,vertical_drop,triple
0,76.0,0,0.0,1,113.0,2,1610.0,1.0,2.0,7,60.0,669.0,0,2,2,550.0,2500,0
1,36.0,4,0.0,0,60.0,0,640.0,2.0,1.0,4,44.0,350.0,0,0,0,0.0,1540,0


In [208]:
X = df_feature3.values
# scale features
X_scale=preprocessing.scale(X)
#train/test split
X_train, X_test, y_train, y_test = train_test_split(X_scale, y_scale, test_size=0.25, random_state=0)
# model fit
lm = linear_model.LinearRegression()
model = lm.fit(X_train,y_train)


In [209]:
# prediction
y_pred=lm.predict(X_test)
# R-squared value
print('r2_score for model 3: ',r2_score(y_test, y_pred))

# mean_absolute_error
print('mean_absolute_error for model 3: ',mean_absolute_error(y_test, y_pred))

print(lm.intercept_)
print(lm.coef_)
print(explained_variance_score(y_test, y_pred))

r2_score for model 3:  0.35216138784224105
mean_absolute_error for model 3:  0.57935685951095
0.006807861660828491
[ 1.39047359e-01  7.24757509e-02  1.11022302e-16 -1.17050678e-01
 -6.63702454e-02  1.53569742e-01 -3.60196149e-02  5.47531613e-02
  2.49049727e-02  8.94467861e-02  4.63587222e-02 -4.55950704e-02
 -5.83410027e-02 -1.54210105e-01  2.18537316e-01  7.62591782e-03
  3.95072531e-01  1.13604871e-01]
0.3529025814166181


In [210]:
df_res3=pd.DataFrame()
df_res3['feature']=df_feature3.columns
df_res3['values']=abs(lm.coef_)
df_res3.sort_values(by=['values'],ascending=False)

,feature,values
16,vertical_drop,3.950725e-01
14,quad,2.185373e-01
13,surface,1.542101e-01
5,fastQuads,1.535697e-01
0,Runs,1.390474e-01
3,trams,1.170507e-01
17,triple,1.136049e-01
9,total_chairs,8.944679e-02
1,double,7.247575e-02
4,Snow Making_ac,6.637025e-02


## Identify the Final Model

**<font color='teal'> Review the model performances in the table below and choose the best model for proving insights to Big Mountain management about what features are driving ski resort lift ticket prices. Type your choice in the final markdown cell. You will explain your selection during the next step of the guided casptone. </font>**

| Model | Explained Variance| Mean Absolute Error|Features Dropped|
| --- | --- | --- | --- |
| Model 1. | -8.48e24| 362070686447 |-|
| Model 2. | 0.3572|0.577 |'state'|
| Model 3. | 0.352 |0.579 |'state','summit_elev','base_elev'|

Model Selection:


- It appears that model 3 has the least 'R^2 score (Explained variance)' and the mean absolute error for model 2 and 3 are very comparable. Model 3 would be the final choice.
- Top three significant features driving the ticket price are 'vertical_drop', 'quad', 'surface'. The management does not have handle on the 'vertical_drop' parameter. Considering this, 'quad', 'surface', 'fastQuads' and 'Runs' should be the most important features to consider.